In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [12]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    while True:
        delta = 0
        # visit all states
        for state in range(env.nS):
            # for each state, we will reset the value and consider all possible actions for that state
            v_state = 0
            for action, action_probability in enumerate(policy[state]):
                # for each action from a state, we will consider all possible next states to calculate its value
                for prob, next_state, reward, done in env.P[state][action]:
                    v_action = prob * (reward + discount_factor * V[next_state])
                # the value of a state will be weighted by the action probabilities
                v_state += action_probability * v_action
            
            delta = max(delta, abs(v_state - V[state]))
            V[state] = v_state 
        if delta < theta:
            break
    return np.array(V)

In [13]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [14]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [15]:
print (v)
print (expected_v)
print (abs(v-expected_v))

[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]
[  0 -14 -20 -22 -14 -18 -20 -20 -20 -20 -18 -14 -22 -20 -14   0]
[0.00000000e+00 6.47062507e-05 9.30238934e-05 1.02390039e-04
 6.47062507e-05 7.94030596e-05 8.62134765e-05 8.52276775e-05
 9.30238934e-05 8.62134765e-05 7.27483887e-05 5.43148418e-05
 1.02390039e-04 8.52276775e-05 5.43148418e-05 0.00000000e+00]
